# Importar os módulos necessários

In [10]:
import warnings
warnings.filterwarnings("ignore")

In [11]:
import pandas as pd
#necessario por causa do metodo de avaliaçao
from sklearn.model_selection import KFold
from sklearn.metrics import (
        make_scorer,
        confusion_matrix, 
        cohen_kappa_score, 
        accuracy_score, 
        precision_score, 
        recall_score, 
        f1_score, 
        roc_auc_score
)
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier # decision trees for classification
from sklearn.neural_network import  MLPClassifier # neural networks for classification
from sklearn.naive_bayes import GaussianNB # naive bayes for classification
from sklearn.svm import SVC # support vector machines for classification

# Definir as Métricas para Avaliação dos Modelos

In [12]:
#metricas regressao
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def custom_regression_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    return mse, mae, r2


In [13]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import make_scorer
import numpy as np

# Definição da função RMSE
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Dicionário de métricas de regressão
REGRESSION_METRICS = {
    "MSE": make_scorer(mean_squared_error),
    "MAE": make_scorer(mean_absolute_error),
    "R2": make_scorer(r2_score),
    "RMSE": make_scorer(root_mean_squared_error)  # Adicionando o RMSE
}



# Ler o Conjunto de Dados

In [14]:
d = pd.read_csv("Cenario3.csv")
d

,mileage,ano,seats,engine_capacity,kms_driven,max_power,resale_price_Lakh
0,23.84000,2019.0,5.0,1199.0,30910.0,83.8,5.66
1,17.50000,2018.0,5.0,1199.0,48089.0,88.7,6.64
2,17.40000,2015.0,5.0,1497.0,51000.0,117.3,5.65
3,14.08000,2021.0,7.0,1956.0,30000.0,167.6,23.00
4,21.40000,2019.0,5.0,1197.0,61113.0,83.1,6.87
...,...,...,...,...,...,...,...
13503,19.49405,2021.0,7.0,2199.0,80000.0,197.2,26.50
13504,19.49405,2017.0,5.0,1197.0,17923.0,81.8,5.87
13505,19.49405,2017.0,5.0,1498.0,63389.0,108.6,7.43
13506,19.49405,2017.0,5.0,1248.0,40000.0,88.5,9.45


In [15]:
X, y = d.drop("resale_price_Lakh", axis=1), d["resale_price_Lakh"]

# Definir o Método de Validação Cruzada

In [16]:
#Classificação
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)

# Utilize KFold para um problema de regressão
kf = KFold(n_splits=10, shuffle=True, random_state=1234)


# Implementação dos Algoritmos de Machine Learning

### Árvores de Decisão

In [17]:
from sklearn.model_selection import KFold, cross_validate
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score



# Definir o regressor e o método de validação cruzada
dt_regressor = DecisionTreeRegressor(max_depth=3, random_state=1234)
kf = KFold(n_splits=10, shuffle=True, random_state=1234)

# Realizar a validação cruzada e avaliar as métricas
scores = cross_validate(dt_regressor, X, y, cv=kf, scoring=REGRESSION_METRICS)
dt_scores = {k: v.mean() for k, v in scores.items()}  # Negar os valores para tornar os scores positivos
pd.DataFrame(dt_scores, index=[0])



,fit_time,score_time,test_MSE,test_MAE,test_R2,test_RMSE
0,0.022901,0.005801,23.475561,2.653967,0.73544,4.77974


### Redes Neuronais

In [18]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_validate

# Criar uma Rede Neural para regressão
nn_regressor = MLPRegressor(hidden_layer_sizes=(50, 50), max_iter=20, random_state=1234)

# Realizar a validação cruzada e avaliar as métricas
scores_nn = cross_validate(nn_regressor, X, y, cv=kf, scoring=REGRESSION_METRICS)
nn_scores = pd.DataFrame(scores_nn)
pd.DataFrame(nn_scores.mean()).T


,fit_time,score_time,test_MSE,test_MAE,test_R2,test_RMSE
0,2.716654,0.005818,46.555589,3.97378,0.463067,6.77172


### Naive Bayes

In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error



# Criar um regressor RandomForest
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=1234)

# Realizar a validação cruzada e avaliar as métricas
scores_rf = cross_validate(rf_regressor, X, y, cv=kf, scoring=REGRESSION_METRICS)
rf_scores = pd.DataFrame(scores_rf)
pd.DataFrame(rf_scores.mean()).T


,fit_time,score_time,test_MSE,test_MAE,test_R2,test_RMSE
0,11.984744,0.064832,10.634483,1.317948,0.885404,3.133816


Random Forest

In [20]:
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

# Gerar dados de exemplo
X, y = make_regression(n_features=4, n_informative=2, random_state=0, shuffle=False)

# Criar um regressor de floresta aleatória
rf_regressor = RandomForestRegressor(max_depth=10, random_state=0)



# Realizar a validação cruzada e avaliar as métricas
scores_rf = cross_validate(rf_regressor, X, y, cv=5, scoring=REGRESSION_METRICS)
rf_scores = pd.DataFrame(scores_rf)
print(pd.DataFrame(rf_scores.mean()).T)
 

   fit_time  score_time   test_MSE  test_MAE   test_R2  test_RMSE
0  0.224195    0.007846  96.022625  7.399889  0.933136   9.583758


### Support Vector Machine